# Qcodes example for National Instruments PXIe-2597 RF Switch

In [1]:
from qcodes_contrib_drivers.drivers.NationalInstruments.PXIe_2597 import NI_PXIe_2597

The PXIe-2597 has a multiplexer configuration. It connects the common `com` port to one of 6 ports. Default channel names are `"ch#"`. We connect to the device using the VISA alias `RF_switch` assigned in NI MAX.

In [2]:
dev = NI_PXIe_2597("Switch", resource="RF_switch")

print('All channels:', [ch.short_name for ch in dev.channels])
print('Initially connected to:',  dev.channel())

Connected to: National Instruments PXI-2597 (serial:1DBFD38, firmware:Not Available) in 6.85s
All channels: ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'com']
Initially connected to: ch3


## Connect different channels on the switch

The main way of connecting channels is the `channel` parameter.

In [3]:
dev.channel("ch3")
print('Now connected to:', dev.channel())

# disconnect by setting to None
dev.channel(None)
print('Now connected to:', dev.channel())

dev.channel("ch1")
print('Now connected to:', dev.channel())

# or by using disconnect_all
dev.disconnect_all()
print('Now connected to:', dev.channel())

Now connected to: ch3
Now connected to: None
Now connected to: ch1
Now connected to: None


Channels may be also connected using their `connect_to` method, although it doesn't make much sense for the PXI-2597, which can have one connection at a time anyway.

In [4]:
dev.channels.ch1.connect_to(dev.channels.com)
print("Connected to:", dev.channel())

Connected to: ch1


You can check which channels are connected to which using the `connections` parameter of a channel. Disconnected channels return an empty list.

In [5]:
dev.channel("ch1")
print("ch1 is connected to", dev.channels.ch1.connections())
print("com is connected to", dev.channels.com.connections())
print("ch2 is connected to", dev.channels.ch2.connections())

ch1 is connected to ['com']
com is connected to ['ch1']
ch2 is connected to []


## Aliasing channel names

You may also provide a dict of aliases for more memorable channel names. This can be also set up in the station YAML file. Note that if you want to use the channel as a setpoint in a measurement, it should be a valid Python identifier (unlike in the example below). The `com` port cannot be aliased.

In [6]:
name_mapping = {'ch1': 'Main channel',
                'ch3': 'Qubit channel'}

dev.close()

dev = NI_PXIe_2597("Switch", resource="RF_switch",
                   name_mapping=name_mapping)

dev.channel('Main channel')
print('Now connected to:', dev.channel())

dev.channel('Qubit channel')
print('Now connected to:', dev.channel())

# non-existent channel throws an error
try:
    dev.channel('Undefined channel')
except ValueError as e:
    print(e)

Connected to: National Instruments PXI-2597 (serial:1DBFD38, firmware:Not Available) in 2.28s
Now connected to: Main channel
Now connected to: Qubit channel
("'Undefined channel' is not in {'ch5', 'ch2', 'ch4', 'Qubit channel', None, 'ch6', 'Main channel'}; Parameter: Switch.channel", 'setting Switch_channel to Undefined channel')


## Print a snapshot

In [7]:
dev.print_readable_snapshot()

Switch:
	parameter value
--------------------------------------------------------------------------------
IDN     :	{'vendor': 'National Instruments', 'model': 'PXI-2597', 'serial': '...
channel :	Qubit channel 
Switch_Main channel:
	parameter  value
--------------------------------------------------------------------------------
connections :	['com'] 
Switch_ch2:
	parameter  value
--------------------------------------------------------------------------------
connections :	[] 
Switch_Qubit channel:
	parameter  value
--------------------------------------------------------------------------------
connections :	[] 
Switch_ch4:
	parameter  value
--------------------------------------------------------------------------------
connections :	[] 
Switch_ch5:
	parameter  value
--------------------------------------------------------------------------------
connections :	[] 
Switch_ch6:
	parameter  value
--------------------------------------------------------------------------------
connecti

In [8]:
dev.close()